# Tool-calling Agent

The agent code implements [MLflow's ChatAgent](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent) interface, a Databricks-recommended open-source standard that simplifies authoring multi-turn conversational agents, and is fully compatible with Mosaic AI agent framework functionality.

**_NOTE:_** This notebook uses LangChain, but AI Agent Framework is compatible with any agent authoring framework, including LlamaIndex or pure Python agents written with the OpenAI SDK.

## Install Dependencies

In [0]:
%pip install uv

In [0]:
%sh uv pip install ../.
%restart_python

In [ ]:
from databricks.connect import DatabricksSession as SparkSession

if "spark" not in locals():
    spark = SparkSession.builder.serverless(True).getOrCreate()

## Agent

Let's test the agent. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

In [0]:
from agent import AGENT
AGENT.predict({"messages": [{"role": "user", "content": "Tell me the current status of Well 19"}]})

In [0]:
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Tell me about Well 19"}]}
):
    print(event, "-----------\n")


## Log the agent as an MLflow model

Determine Databricks resources to specify for automatic auth passthrough at deployment time
- **TODO**: If your Unity Catalog Function queries a [vector search index](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/unstructured-retrieval-tools) or leverages [external functions](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/external-connection-tools), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#specify-resources-for-automatic-authentication-passthrough) for more details.

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).


In [0]:
import mlflow
from agent import LLM_ENDPOINT_NAME, tools
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

mlflow.set_registry_uri("databricks-uc")

uc_model_name = f"{config.catalog}.{config.schema}.{config.agentify.agent_name}"

resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksServingEndpoint(endpoint_name='shm_3w_lightgbm')
    ]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "What is the current status of Well 19?"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        registered_model_name=uc_model_name,
        pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            f"mlflow=={get_distribution('mlflow').version}",
            f"databricks-langchain=={get_distribution('databricks-langchain').version}",
            f"langgraph=={get_distribution('langgraph').version}",
        ],
    )

In [0]:
from mlflow.deployments import get_deploy_client
from databricks import agents

agents.deploy(
  uc_model_name, 
  logged_agent_info.registered_model_version, 
  tags={"endpointSource": "playground"}
)